In [ ]:
# from ipynb.fs.full.common import *
%run common.ipynb

Variables

In [ ]:
# # ['Hemlata', 'Malti', 'Preeti', 'Sharifa', 'Vinita', 'VKS'] ['Geeta', 'Jitendra', 'Jyoti', 'Kuldeep', 'Seema', 'VijayLaxmi']
# ACTIVE_SHAM = 'Active'
# SAMPLE = 'Hemlata'
# ELECTRODES = '32electrodes' # '32electrodes' 
GROUP1 = 'Pre'
GROUP2 = 'Post' # generally GROUP1+1
mapping = {0: 'theta', 1: 'alpha', 2: 'beta'}

Importing all packages

In [ ]:
# import numpy as np
# import pandas as pd
# import mne
# import matplotlib
# import matplotlib.pyplot as plt
# import os
# from scipy.stats import ttest_rel
# from mne_connectivity import spectral_connectivity_epochs
# import networkx as nx
# import numpy as np 
# import glob

# matplotlib.use('TkAgg')

## LOAD DATA

In [ ]:
# # 2Pre-TDCS : .easy, # 3TDCS : .easy, # 4Post-TDCS : .easy
# import glob
# folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES, ACTIVE_SHAM)
# # folders = ['1Pre-BML', '2Pre-TDCS', '3TDCS', '4Post-TDCS', '5Post-BML']
# # folders = ['2Pre-TDCS', '3TDCS', '4Post-TDCS']
# all_files = []
# # for folder in folders:
# #     ext = '.eeg' if folder[0] in ['1', '5'] else 'Close.easy'
# #     is_even = 1
# #     for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
# #         if (folder[0] == '3' and is_even%2==1) or folder[0] in ['2', '4']: #TODO: add for both 1 and 5 also 
# #             all_files.append(_)
# #         is_even += 1

# #     sorted_files = sorted(all_files, key=lambda x: os.path.getmtime(os.path.join(folder_path, folder, SAMPLE, x)))
# # print(len(sorted_files), sorted_files)
# folders = ['2Pre-TDCS', '4Post-TDCS']
# for folder in folders:
#     ext = 'Close.easy'
#     for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
#         all_files.append(_)

# print(len(all_files), all_files)

# # TODO: ADD iterator that changes Groups value and iterates (len(sorted_files) - 1 times)
# raw_1 = data_transformation_easy(all_files[0])
# raw_1 = g1_preprocess(raw_1)
# raw_2 = data_transformation_easy(all_files[1])
# raw_2 = g1_preprocess(raw_2)

In [ ]:
folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES, ACTIVE_SHAM)
all_files = []

folders = ['2Pre-TDCS', '4Post-TDCS']
for folder in folders:
    ext = 'Close.easy' if ELECTRODES == '32electrodes' else 'EEG.easy'
    for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
        all_files.append(_)

# TODO: ADD iterator that changes Groups value and iterates (len(sorted_files) - 1 times)
raw_1 = data_transformation_easy(all_files[0])
# SHOW=True
# fig = raw_1.plot(
#         n_channels=32, 
#         scalings=SCALINGS,
#         show=SHOW
#         )
# info = raw_1.info

# from scipy.io import loadmat
# matfiles = glob.glob(os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Preeti_PRE_clean.mat'))
# data = {}
# print(matfiles)
# dat = loadmat(matfiles[0])
# raw_2 = np.array(dat['EEG']['data'][0][0])
# raw_2 = mne.io.RawArray(raw_2, info)
# fig = raw_2.plot(
#         n_channels=32, 
#         scalings=2e2,
#         show=SHOW
#         )

raw_1 = g1_preprocess(raw_1)
raw_2 = data_transformation_easy(all_files[1])
raw_2 = g1_preprocess(raw_2)

Excel data

In [ ]:
existing_file = f"comparison-results/result-2-{ACTIVE_SHAM}.xlsx"
df = pd.read_excel(existing_file, header=[0,1])

# main_column_names = ['index', 'swn', 'links', 'asymmetry', 'hubs', 'channels', 'regions', 'cc', 'lavg', 'eglo', 'eloc']
# subcols = ('theta-g1', 'theta-g2', 'alpha-g1', 'alpha-g2', 'beta-g1', 'beta-g2')

# subcolumn_names = {
#     'index' : ['index'],
#     'swn': subcols,
#     'links': subcols,
#     'asymmetry': subcols,
#     'hubs': ('theta', 'alpha', 'beta'),
#     'channels': ('theta', 'alpha', 'beta'),
#     'regions': ('theta', 'alpha', 'beta'),
#     'cc' : subcols,
#     'lavg': subcols,
#     'eglo': subcols,
#     'eloc': subcols
# }

# columns_tuples = [(main_col, sub_col) for main_col in main_column_names for sub_col in subcolumn_names[main_col]]
# columns = pd.MultiIndex.from_tuples(columns_tuples)
# df = pd.DataFrame(columns=columns)

row_data = {('index', 'index'): f'{ACTIVE_SHAM}-{SAMPLE}-{GROUP1}vs{GROUP2}'}
df

### 1. PSD graph plot between MDD and Control group

For resting state EEG channel, analyzing the overall average power across channels provides a holistic view of the brain's activity without emphasizing the specificity of individual channels.

1.1 PSD using plot_psd

In [ ]:
fig, ax = plt.subplots()
#plot_psd uses welch method for continuous data
raw_1.plot_psd(picks=raw_1.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='blue', dB=False)
raw_2.plot_psd(picks=raw_2.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='red', dB=False)

# dB = True plots PSD in decibels (logarithmic)
# different n_fft compared to psd_array_welch
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power/Frequency (microV^2/Hz)')
ax.set_title('Power Spectral Density Comparison')

ax.text(0.8, 0.9, GROUP1, color='blue', transform=ax.transAxes)
ax.text(0.8, 0.85, GROUP2, color='red', transform=ax.transAxes)

plt.ylim(0, 8)
# plt.show()

fig.savefig(f'comparison-results/PSD - {SAMPLE} - {GROUP1} vs {GROUP2}.png')

### 2. Epoching

In [ ]:
duration = 4.0
overlap = 2.0 

samples_per_epoch = int(duration * SFREQ)
samples_per_overlap = int(overlap * SFREQ)

# Manually created events
start, stop = 0, samples_per_epoch
events = []
while stop <= len(raw_1):
    events.append([start, 0, 1]) 
    start += samples_per_overlap
    stop += samples_per_overlap

events = np.array(events)

2.1 Segmenting epochs for Group 1

In [ ]:
epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_1.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_1.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

In [ ]:
# raw_1.plot()
# ica = ICA(n_components=20, method='fastica', random_state=23).fit(raw_1)
# # ica.exclude = [1]
# # raw_clean = ica.apply(raw.copy())
# ica.plot_components()
# ica.plot_properties(raw_1, picks=14)

In [ ]:
epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_2.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

In [ ]:
# import autoreject

# ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,n_jobs=1, verbose=True)
# ar.fit(epochs_1[:20])  # fit on a few epochs to save time
# epochs_1, reject_log = ar.transform(epochs_1, return_log=True)


# ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,n_jobs=1, verbose=True)
# ar.fit(epochs_2[:20])  # fit on a few epochs to save time
# epochs_2, reject_log = ar.transform(epochs_2, return_log=True)

In [ ]:
# Amplitude plots between both groups
epochs_1_avg = epochs_1['1'].average()
epochs_2_avg = epochs_2['1'].average()
# evokeds = dict(epochs_1=epochs_1)
# fig_MDD = epochs_1_avg.plot(titles=GROUP1, time_unit = 'ms')
# fig_control = epochs_2_avg.plot(titles=GROUP2, time_unit = 'ms')

### 3. PLI and construction of brain function matrices

In [ ]:
# pli method always gives positive correlations
connectivity = {}
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=FREQ_BANDS[mapping[i]][0], fmax=FREQ_BANDS[mapping[i]][1], faverage=True, n_jobs=1)


In [ ]:
n_channels = len(raw_1.info['ch_names'])
for i in mapping:
    con = connectivity_matrix(epochs_1, i)
    connectivity[f'{GROUP1}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    con = connectivity_matrix(epochs_2, i)
    connectivity[f'{GROUP2}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))


### 3. Thresholding - M1 -  Small World Index

Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [ ]:
# def create_random_graph(n, e):
#     adj_matrix = np.zeros((n, n))
#     edges = [(i, j) for i in range(n) for j in range(i) if i != j]
#     selected_edges = np.random.choice(len(edges), e, replace=False)
#     for edge_idx in selected_edges:
#         i, j = edges[edge_idx]
#         adj_matrix[i, j] = 1
#     return adj_matrix 

In [ ]:
def make_symmetric(arr, n_channels):
    for row in range(n_channels):
        for col in range(n_channels):
            arr[row][col] = arr[col][row]
    return arr

In [ ]:
def calculate_swn(mat, threshold):
    tthresh = np.median(mat[mat!=0]) + threshold*np.std(mat[mat!=0])
    print("*****111111******", tthresh)
    binmat= mat > tthresh
    n_connections = np.count_nonzero(binmat == 1)
    print("*****222222******", n_connections)
    binmat = make_symmetric(binmat.astype(int), n_channels)
    g= calculate_avergae_components(nx.from_numpy_array(binmat))
    Lw_binarized, CC_binarized = g[0], g[3]
    return (CC_binarized/Lw_binarized if n_connections else 0), n_connections

In [ ]:
# WINDOW_SIZE = 5
# def moving_average(data, window_size = WINDOW_SIZE):
#     cumsum = np.cumsum(data, dtype=float)
#     cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
#     return cumsum[window_size - 1:] / window_size

In [ ]:
import matplotlib.pyplot as plt
import random

n_perms = 1000
n_network = n_perms//10
# thresholds = np.linspace(0, 1.5, 200)
swn_z_1, swn_z_2 = {}, {}
# optimal_threshold = {}
optimal_swn_perms, optimal_swn_real1, optimal_swn_real2 = {}, {}, {}
# moving_avg1, moving_avg2 = {}, {}

for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    # swn_z_1[mapping[i]], swn_z_2[mapping[i]] = 0, 0
    # optimal_swn_perms[mapping[i]], optimal_swn_real1[mapping[i]], optimal_swn_real2[mapping[i]] = [[] for _ in range(len(thresholds))], np.zeros(len(thresholds)),  np.zeros(len(thresholds))
    # max_diff = 0
    if ACTIVE_SHAM == 'Active':
        x = {'theta': 0.9422110552763826, 'alpha': 0.4824120603015075, 'beta': 1.2738693467336675}
    else:
        x = {'theta': 0.2788944723618091, 'alpha': 0.391959798994975, 'beta': 0.2638190954773868}
    threshold = x[mapping[i]]
    # for threshi, threshold in enumerate(thresholds):
    #     print("00000000", threshi, threshold)
    swn_1, n_connections1 = calculate_swn(conmat1, threshold)
    swn_2, n_connections2 = calculate_swn(conmat2, threshold)

    # Create #n_network random matrices with vertices: n_channels and edges: average of edges of thresholded binary matrix of both groups 
    # And make a list of #n_network lists of clustering coeffs and shortest avg path length
    # Crand, Lrand = [], []
    # swn_perms = np.zeros(n_perms)
    # for perm in range(n_network):
    #     random_binmat = create_random_graph(n_channels, (n_connections1+n_connections2)//2)
    #     random_binmat = make_symmetric(random_binmat, n_channels)
    #     grand = calculate_avergae_components(nx.from_numpy_array(random_binmat))
    #     Crand.append(grand[3])
    #     Lrand.append(grand[0])

    # print("CHECK", Crand.count(0))
    # print("CHECK2", Lrand.count(0))
    # # choose any 2 random network from above n_networks #n_perms times 
    # for perm in range(n_perms):
    #     whichnetworks2use = random.sample(range(n_network), 2)
    #     if Crand[whichnetworks2use[1]] and Lrand[whichnetworks2use[0]]:
    #         swn_perms[perm] = (Crand[whichnetworks2use[0]] / Crand[whichnetworks2use[1]]) / (Lrand[whichnetworks2use[0]] / Lrand[whichnetworks2use[1]])
    #     else:
    #         perm -= 1
    
    # swn_rand_avg = np.mean(Crand, axis=0)/np.mean(Lrand, axis=0)
    # swn_real_1 = swn_1 / swn_rand_avg # TODO: different random avg or same ?
    # swn_real_2 = swn_2 / swn_rand_avg
    # val1, val2 = (swn_real_1 - np.mean(swn_perms)) / np.std(swn_perms), (swn_real_2 - np.mean(swn_perms)) / np.std(swn_perms)
    # swn_z_1[mapping[i]] = val1
    # swn_z_2[mapping[i]] = val2

    # max_diff = 0
    # moving_avg1[mapping[i]] = moving_average(swn_z_1[mapping[i]])
    # moving_avg2[mapping[i]] = moving_average(swn_z_2[mapping[i]])
    # moving_threshold = moving_average(thresholds)

    # print("$$$$$", optimal_swn_perms[mapping[i]])
    # optimal_swn_perms[mapping[i]] = swn_perms
    row_data[('swn', f'{mapping[i]}-g1')] = swn_1 
    row_data[('swn', f'{mapping[i]}-g2')] = swn_2 

In [ ]:
# # High standardized network metric suggests that our network exhibits properties away from random networks 
# for i in mapping:
#     plt.plot(thresholds, swn_z_1[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP1}')
#     plt.plot(thresholds, swn_z_2[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP2}')
#     plt.xlabel(f'Threshold (Standard deviations above median) at frequency band {mapping[i]}')
#     plt.ylabel(f'SWN_z')
#     plt.xlim([0, 1.5])
#     plt.legend()
#     # plt.show()

### 4. Binarization of functional connectivity

In [ ]:
# # To maximize the statistical significance of small world networkness for both groups while 
# # maximizing the difference between the z-normalized small world networks of the two groups.
# f, axarr = plt.subplots(1,3, figsize=(100, 5)) 
# for count, i in enumerate(mapping):
#     max_diff = 0
#     for mov in range(len(moving_threshold)):
#         x = np.abs(moving_avg1[mapping[i]][mov] - moving_avg2[mapping[i]][mov])
#         if x>max_diff:
#             max_diff = x
#             optimal_threshold[mapping[i]] = moving_threshold[mov]
#             index = mov

#     # TODO: save fig file
#     p_value_1 = np.mean(optimal_swn_perms[mapping[i]][index+(WINDOW_SIZE//2)] >= optimal_swn_real1[mapping[i]][index+(WINDOW_SIZE//2)])
#     p_value_2 = np.mean(optimal_swn_perms[mapping[i]][index+(WINDOW_SIZE//2)] >= optimal_swn_real2[mapping[i]][index+(WINDOW_SIZE//2)])
#     print(mapping[i], "p-value: ", p_value_1, p_value_2)


#     axarr[count].hist(optimal_swn_perms[mapping[i]][index+(WINDOW_SIZE//2)], bins=50, color='b', alpha=0.7)
#     axarr[count].axvline(optimal_swn_real1[mapping[i]][mov+(WINDOW_SIZE//2)], color='m', linewidth=3, label=f'{GROUP1}')
#     axarr[count].axvline(optimal_swn_real2[mapping[i]][mov+(WINDOW_SIZE//2)], color='r', linewidth=3, label=f'{GROUP2}')
#     axarr[count].set_xlabel(f'{mapping[i]}')
#     axarr[count].set_xlim([0,10])
#     # axarr[count].set_ylim([0,100])
#     axarr[count].legend()
# f.suptitle('Random Network distribution for both Pre and Post groups')
# # plt.show()

In [ ]:
# Find optimal threhsold
binarized_matrix = {}
print(x[mapping[i]])
for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    tthresh1 = np.median(conmat1[conmat1!=0]) + x[mapping[i]]*np.std(conmat1[conmat1!=0])
    tthresh2 = np.median(conmat2[conmat2!=0]) + x[mapping[i]]*np.std(conmat2[conmat2!=0])
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > tthresh1
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > tthresh2
    print(tthresh1, tthresh2)
print(binarized_matrix.keys())


Find number of links in each graph

In [ ]:
for i in mapping:
    links_1 = 0
    links_2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_2)

    row_data[('links', f'{mapping[i]}-g1')] = links_1
    row_data[('links', f'{mapping[i]}-g2')] = links_2

### 5. Asymmetry in  each band

In [ ]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

for i in mapping:
    lr = 1
    rr = 1
    lrc = 1
    rrc = 1
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

    row_data[('asymmetry', f'{mapping[i]}-g1')] = lr/rr
    row_data[('asymmetry', f'{mapping[i]}-g2')] = lrc/rrc

In [ ]:
f, axarr = plt.subplots(1,3) 
for count, i in enumerate(mapping):
    conmat1 = make_symmetric(connectivity[f'{GROUP1}-{mapping[i]}'], n_channels)
    # conmat2 = make_symmetric(connectivity[f'{GROUP2}-{mapping[i]}'], n_channels)

    im = axarr[count].imshow(conmat1)
    axarr[count].set_title(f'Functional connectivity Pre {mapping[i]} in {ACTIVE_SHAM} treatment')
    axarr[count].set_xticks([_ for _ in range(n_channels)])
    axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)])
    axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

f.text(0.5, 0.04, 'X-axis label', ha='center')
f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')

# plt.show()

In [ ]:
f, axarr = plt.subplots(1,3) 
for count, i in enumerate(mapping):
    # conmat1 = make_symmetric(connectivity[f'{GROUP1}-{mapping[i]}'], n_channels)
    conmat2 = make_symmetric(connectivity[f'{GROUP2}-{mapping[i]}'], n_channels)

    im = axarr[count].imshow(conmat1)
    axarr[count].set_title(f'Functional connectivity Post {mapping[i]} in {ACTIVE_SHAM} treatment')
    axarr[count].set_xticks([_ for _ in range(n_channels)])
    axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)])
    axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

f.text(0.5, 0.04, 'X-axis label', ha='center')
f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')

# plt.show()

### 6. Difference matrix

In [ ]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [ ]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [ ]:
difference_matrix = {}
for i in mapping:
    # TODO: both difference and excel it in in different column
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int)
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    difference_matrix[mapping[i]] = make_symmetric(difference_matrix[mapping[i]], n_channels)

    # # Yellow - Higher synchronisation
    # diff_mat = difference_matrix[mapping[i]]
    # plt.imshow(diff_mat, interpolation='none')
    # plt.title(f'Binary FC difference in {ACTIVE_SHAM} treatment in {mapping[i]}')
    # plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    # plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    # plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    # plt.show()

    
    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

In [ ]:
from matplotlib import cm, colors
f, axarr = plt.subplots(1,3) 
for count, i in enumerate(mapping):
    conmat1 = make_symmetric((connectivity[f'{GROUP2}-{mapping[i]}'] - connectivity[f'{GROUP1}-{mapping[i]}']), n_channels)

    im = axarr[count].imshow(conmat1)
    axarr[count].set_title(f'FCdifferent maaatrix {mapping[i]} in {ACTIVE_SHAM} treatment')
    axarr[count].set_xticks([_ for _ in range(n_channels)])
    axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
    axarr[count].set_yticks([_ for _ in range(n_channels)])
    axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
    axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

f.colorbar(cm.ScalarMappable(norm=colors.Normalize(0, 5), cmap=cm.get_cmap("Spectral")), ax=ax)
f.text(0.5, 0.04, 'X-axis label', ha='center')
f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')

# plt.show()

In [ ]:
def calculate_hub(mat):
        hub =[]
        for ch, ch_name in enumerate(raw_1.info['ch_names']):
            hub.append(np.count_nonzero(mat[ch] == 1))
        return hub
        # layout = find_layout(raw_1.info)
        # plot_topomap(hub, layout.pos, names=raw_1.info['ch_names'], sphere='eeglab')
        # plt.show()

In [ ]:
edges_threshold = {}
for i in mapping:
    hub = calculate_hub(difference_matrix[mapping[i]])
    edges_threshold[mapping[i]] = np.median(hub) + np.std(hub)
    row_data[('hubs', mapping[i])] = hub
print(edges_threshold)

In [ ]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []
    for ch in range(n_channels):
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > edges_threshold[mapping[i]]:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

Create dictionary of Regions in brain vs channels

In [ ]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [ ]:
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)

In [ ]:
for i in mapping:
    row_data[('channels', mapping[i])] = selected_channels[mapping[i]]

In [ ]:
for i in mapping:
    row_data[('regions', mapping[i])] = selected_regions[mapping[i]]
row_data

ADD EXTRAS IN EXCEL FILE

In [ ]:
for i in mapping:
    g1 = calculate_avergae_components(nx.from_numpy_array(binarized_matrix[f'{GROUP2}-{mapping[i]}']))
    g2 = calculate_avergae_components(nx.from_numpy_array(binarized_matrix[f'{GROUP2}-{mapping[i]}']))

    row_data[('cc', f'{mapping[i]}-g1')] = g1[3]
    row_data[('cc', f'{mapping[i]}-g2')] = g2[3]
    row_data[('lavg', f'{mapping[i]}-g1')] = g1[0]
    row_data[('lavg', f'{mapping[i]}-g2')] = g2[0]
    row_data[('nbc', f'{mapping[i]}-g1')] = g1[1]
    row_data[('nbc', f'{mapping[i]}-g2')] = g2[1]
    row_data[('eglo', f'{mapping[i]}-g1')] = g1[2]
    row_data[('eglo', f'{mapping[i]}-g2')] = g2[2]
    row_data[('eloc', f'{mapping[i]}-g1')] = g1[4]
    row_data[('eloc', f'{mapping[i]}-g2')] = g2[4]

In [ ]:
# for i in mapping:
#     g1 = calculate_avergae_components(nx.from_numpy_array(connectivity_matrix[f'{GROUP1}-{mapping[i]}']))
#     g2 = calculate_avergae_components(nx.from_numpy_array(connectivity_matrix[f'{GROUP2}-{mapping[i]}']))

#     row_data[('swn', f'{mapping[i]}-g1')] = g1[3]/g1[0]
#     row_data[('swn', f'{mapping[i]}-g2')] = g2[3]/g2[0]
#     row_data[('cc', f'{mapping[i]}-g1')] = g1[3]
#     row_data[('cc', f'{mapping[i]}-g2')] = g2[3]
#     row_data[('lavg', f'{mapping[i]}-g1')] = g1[0]
#     row_data[('lavg', f'{mapping[i]}-g2')] = g2[0]
#     row_data[('nbc', f'{mapping[i]}-g1')] = g1[1]
#     row_data[('nbc', f'{mapping[i]}-g2')] = g2[1]
#     row_data[('eglo', f'{mapping[i]}-g1')] = g1[2]
#     row_data[('eglo', f'{mapping[i]}-g2')] = g2[2]
#     row_data[('eloc', f'{mapping[i]}-g1')] = g1[4]
#     row_data[('eloc', f'{mapping[i]}-g2')] = g2[4]
#     row_data[('hub', f'{mapping[i]}-g1')] = g1[5]
#     row_data[('hub', f'{mapping[i]}-g2')] = g2[5]

In [ ]:
print(row_data, len(df))

In [ ]:
# df = df.drop(df.index)
new_row = pd.Series(row_data)
print("length", len(df))
print("new row", new_row)
df.loc[len(df)] = new_row

In [ ]:
(df)

In [ ]:
# # Reset the column index
df.to_excel(existing_file, index=True, header=True)
df